In [6]:
import numpy as np
import pandas as pd
import mase
import random
import typing
import plotnine
import tqdm

In [15]:
def test_pathfind():
    results = list()
    for map_size in tqdm.tqdm(range(5, 30)):
        for percent_avoid in [0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
            diffs = list()
            for i in range(500):
                start, target, avoidset = mase.hexmapgenerator.random_target_map(map_size, percent_avoid=percent_avoid)
                sp_len = start.shortest_path_length(target, avoidset)
                if sp_len is not None:
                    found_path = start.pathfind_dfs(target, avoidset)
                    assert(len(found_path) >= sp_len)
                    diff = len(found_path) - sp_len
                    diffs.append(diff)
            
                results.append({
                    'map_size': map_size, 
                    'percent_avoid': percent_avoid, 
                    'i': i,
                    'diff': diff, 
                    #'median': np.median(diffs), 
                    #'mean': np.mean(diffs)
                })
                            
    return pd.DataFrame(results)
results = test_pathfind()

100%|██████████| 25/25 [17:11<00:00, 41.26s/it] 


In [49]:
aggdf = results.groupby(['map_size', 'percent_avoid']).agg(
    av_diff = pd.NamedAgg(column='diff', aggfunc=np.mean),
    med_diff = pd.NamedAgg(column='diff', aggfunc=np.median),
).reset_index()

(plotnine.ggplot(aggdf) + 
    plotnine.aes(x='map_size', y='med_diff', group='percent_avoid') + 
    plotnine.geom_line(show_legend=True)
)
(results['diff'] > 5).sum(), results.shape[0]

(17500, 75000)

In [ ]:
(plotnine.ggplot() + 
    plotnine.aes(x='map_size', y='diff', group='percent_avoid') + 
    plotnine.geom_violin(show_legend=True)
)

In [3]:
mase.CyHexPosition(0, 0, 0).dist(mase.CyHexPosition(-3, -2, 5))

5.0

In [7]:
def test_data(hmap, seed: int):
    random.seed(seed)
    all_positions = hmap.positions()
    avoidset = set(random.sample(all_positions, len(all_positions) // 4))
    start = random.choice(list(all_positions))
    end = start
    while end == start:
        end = random.choice(list(all_positions))
    return start, end, avoidset

def run_test(map_size: int, num_runs: int):
    hmap = first.HexMap(map_size)
    path_lengths = list()
    for i in range(num_runs):
        start, end, avoidset = test_data(hmap, i)
        path = start.shortest_path_dfs(end, avoidset, 2*map_size)
        path_lengths.append(len(path))
    return start, end, sum(pl for pl in path_lengths if pl is not None)

%timeit run_test(100, 100)

1.07 s ± 10.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
